# Upload datasets

In [1]:
import pandas as pd
import numpy as np
import sys
import math

In [2]:
# Load search trends dataset
st_url = 'https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_daily_symptoms_dataset.csv'
df1 = pd.read_csv(st_url)
print(df1)


      open_covid_region_code  ... symptom:pancreatitis
0                      US-AK  ...                  NaN
1                      US-AK  ...                  NaN
2                      US-AK  ...                  NaN
3                      US-AK  ...                 0.82
4                      US-AK  ...                  NaN
...                      ...  ...                  ...
14785                  US-WY  ...                 0.59
14786                  US-WY  ...                  NaN
14787                  US-WY  ...                  NaN
14788                  US-WY  ...                  NaN
14789                  US-WY  ...                  NaN

[14790 rows x 430 columns]


In [3]:
# Load Hospitalization dataset
hp_url = 'https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv'
df2 = pd.read_csv(hp_url,  dtype={'test_units': str})
print(df2)


       open_covid_region_code  ... economic_support_index_for_display
0                         AFG  ...                                NaN
1                         AFG  ...                                0.0
2                         AFG  ...                                0.0
3                         AFG  ...                                0.0
4                         AFG  ...                                0.0
...                       ...  ...                                ...
102907                    VUT  ...                               25.0
102908                    VUT  ...                               25.0
102909                    VUT  ...                               25.0
102910                    VUT  ...                               25.0
102911                    VUT  ...                               25.0

[102912 rows x 62 columns]


# Remove Empty Columns

In [4]:
# Remove empty columns for search trends dataset 
def is_unique(col): #to check if all elements in a column are the same
    col_np = col.to_numpy() 
    return (col_np[0] == col_np).all() #return a boolean

df1_temp = df1
df1_bool = df1.isnull() #check if a dataframe contains NaN, return an array of bool
count = 0
for col in df1_bool.iteritems():
  if col[1][0] == True and is_unique(col[1]) == True: #check if the entire column is NaN
    df1_temp = df1_temp.drop(col[0], 1)
print(df1_temp)

# replace NAN with zero value 
df1_temp = df1_temp.fillna(0)


      open_covid_region_code  ... symptom:pancreatitis
0                      US-AK  ...                  NaN
1                      US-AK  ...                  NaN
2                      US-AK  ...                  NaN
3                      US-AK  ...                 0.82
4                      US-AK  ...                  NaN
...                      ...  ...                  ...
14785                  US-WY  ...                 0.59
14786                  US-WY  ...                  NaN
14787                  US-WY  ...                  NaN
14788                  US-WY  ...                  NaN
14789                  US-WY  ...                  NaN

[14790 rows x 428 columns]


# Get All Common Regions

In [5]:
# Extract data for US only 
df2_temp = df2
df2_temp = df2_temp[df2_temp['open_covid_region_code'].str.startswith('US-',na = False)] 
print(df2_temp)

      open_covid_region_code  ... economic_support_index_for_display
83900                  US-WY  ...                                NaN
83901                  US-WY  ...                                NaN
83902                  US-WY  ...                                NaN
83903                  US-WY  ...                                NaN
83904                  US-WY  ...                                NaN
...                      ...  ...                                ...
96716                  US-AK  ...                                NaN
96717                  US-AK  ...                                NaN
96718                  US-AK  ...                                NaN
96719                  US-AK  ...                                NaN
96720                  US-AK  ...                                NaN

[12821 rows x 62 columns]


In [6]:
# Deal with inconsistent regions in two datasets

df1_regions = df1_temp['open_covid_region_code'].unique() #get the column containing unique(non-repetitive) region names
df2_regions = df2_temp['open_covid_region_code'].unique()
df2_temp = df2_temp.set_index('open_covid_region_code') # set region codes as the indexes of dataframe 2
diff = list(set(df2_regions) - set(df1_regions) & set(df2_regions)) #find the regions in dataset 2 that are not in dataset 1

In [7]:
# Refine dataset 2 (hospitalization) 
df2_temp.drop(index=diff, axis = 0, inplace = True) #remove unwanted regions in dataset 2 
print(df2_temp)

df2_temp = df2_temp[['region_name', 'date','hospitalized_new']] # Extract columns 'region_name', 'date', and 'hospitalization_new' in dataset 2

df2_temp = df2_temp.fillna(0) # Replace all NaN with zero 
print(df2_temp)

                       region_name  ... economic_support_index_for_display
open_covid_region_code              ...                                   
US-WY                      Wyoming  ...                                NaN
US-WY                      Wyoming  ...                                NaN
US-WY                      Wyoming  ...                                NaN
US-WY                      Wyoming  ...                                NaN
US-WY                      Wyoming  ...                                NaN
...                            ...  ...                                ...
US-AK                       Alaska  ...                                NaN
US-AK                       Alaska  ...                                NaN
US-AK                       Alaska  ...                                NaN
US-AK                       Alaska  ...                                NaN
US-AK                       Alaska  ...                                NaN

[11736 rows x 61 columns

In [8]:
# Convert to Numpy Array
regions = df1_temp['open_covid_region_code'].unique()
df2_temp = df2_temp.reset_index() # switch back to default indexing of dataframe 2
print(df2_temp)

df1_array = df1_temp.values # convert pandas dataframe to numpy array
df2_array = df2_temp.values 
print(df2_array)
print(type(df1_array[0]), type(df2_array[0]))


      open_covid_region_code region_name        date  hospitalized_new
0                      US-WY     Wyoming  2020-03-07               0.0
1                      US-WY     Wyoming  2020-03-08               0.0
2                      US-WY     Wyoming  2020-03-09               0.0
3                      US-WY     Wyoming  2020-03-10               0.0
4                      US-WY     Wyoming  2020-03-11               0.0
...                      ...         ...         ...               ...
11731                  US-AK      Alaska  2020-10-14               0.0
11732                  US-AK      Alaska  2020-10-15               0.0
11733                  US-AK      Alaska  2020-10-16               0.0
11734                  US-AK      Alaska  2020-10-17               0.0
11735                  US-AK      Alaska  2020-10-18               0.0

[11736 rows x 4 columns]
[['US-WY' 'Wyoming' '2020-03-07' 0.0]
 ['US-WY' 'Wyoming' '2020-03-08' 0.0]
 ['US-WY' 'Wyoming' '2020-03-09' 0.0]
 ...
 ['

In [9]:
# Re-organize regions for both datasets with the same order
df1_region_array = [] # create numpy arrays to hold the re-ordered dataset 1
df2_region_array = [] # create numpy arrays to hold the re-ordered dataset 2
for state in regions: 
  region_array_st = [] # created numpy arrays to hold the re-ordered subset for each region in dataset 1
  region_array_hp = [] # created numpy arrays to hold the re-ordered subset for each region in dataset 2
  
  for row in df1_array: 
    if state == row[0]:
      region_array_st.append(row)
  df1_region_array.append(region_array_st)

  for row in df2_array: 
    if state == row[0]:
      region_array_hp.append(row)
  df2_region_array.append(np.array(region_array_hp))

df1_region_array = np.array(df1_region_array)
df2_region_array = np.array(df2_region_array)
print(df1_region_array.shape)
print(df2_region_array.shape)
print(df2_region_array[0].shape)

(51, 290, 428)
(51,)
(227, 4)


In [10]:
# Convert back to dataframe 
x, y, z = df1_region_array.shape
df1_region_array = np.reshape(df1_region_array, (x*y, z))

print(df1_region_array.shape)
print(df1_region_array)

expand_list = []
for i in df2_region_array:
  for j in i: 
    expand_list.append(j)

df2_region_array = np.array(expand_list)
print(df2_region_array.shape)
print(df2_region_array)


(14790, 428)
[['US-AK' 'US' 'United States' ... 0.0 0.81 0.0]
 ['US-AK' 'US' 'United States' ... 0.0 0.0 0.0]
 ['US-AK' 'US' 'United States' ... 0.0 0.87 0.0]
 ...
 ['US-WY' 'US' 'United States' ... 0.0 0.47 0.0]
 ['US-WY' 'US' 'United States' ... 0.0 0.0 0.0]
 ['US-WY' 'US' 'United States' ... 0.0 0.0 0.0]]
(11736, 4)
[['US-AK' 'Alaska' '2020-03-06' 0.0]
 ['US-AK' 'Alaska' '2020-03-07' 0.0]
 ['US-AK' 'Alaska' '2020-03-08' 0.0]
 ...
 ['US-WY' 'Wyoming' '2020-10-16' 5.0]
 ['US-WY' 'Wyoming' '2020-10-17' -1.0]
 ['US-WY' 'Wyoming' '2020-10-18' 2.0]]


In [11]:
# Convert back to dataframe
df1_column = df1_temp.columns.values
df1_temp = pd.DataFrame(df1_region_array, columns=df1_column)
print(df1_temp)

df2_column = df2_temp.columns.values
df2_temp = pd.DataFrame(df2_region_array, columns=df2_column)
print(df2_temp)

      open_covid_region_code  ... symptom:pancreatitis
0                      US-AK  ...                    0
1                      US-AK  ...                    0
2                      US-AK  ...                    0
3                      US-AK  ...                 0.82
4                      US-AK  ...                    0
...                      ...  ...                  ...
14785                  US-WY  ...                 0.59
14786                  US-WY  ...                    0
14787                  US-WY  ...                    0
14788                  US-WY  ...                    0
14789                  US-WY  ...                    0

[14790 rows x 428 columns]
      open_covid_region_code region_name        date hospitalized_new
0                      US-AK      Alaska  2020-03-06                0
1                      US-AK      Alaska  2020-03-07                0
2                      US-AK      Alaska  2020-03-08                0
3                      US-AK    

# Reorganize date 

In [12]:
# Reorganize date 

# search trends dataset start date are 2020-01-06 Monday 
# Hospitalization has different start date. We decide to start at 2020-03-09 Monday 
df1_temp = df1_temp.set_index('date')
df2_temp = df2_temp.set_index('date')
print(df1_temp)
print(df2_temp)

           open_covid_region_code  ... symptom:pancreatitis
date                               ...                     
2020-01-01                  US-AK  ...                    0
2020-01-02                  US-AK  ...                    0
2020-01-03                  US-AK  ...                    0
2020-01-04                  US-AK  ...                 0.82
2020-01-05                  US-AK  ...                    0
...                           ...  ...                  ...
2020-10-12                  US-WY  ...                 0.59
2020-10-13                  US-WY  ...                    0
2020-10-14                  US-WY  ...                    0
2020-10-15                  US-WY  ...                    0
2020-10-16                  US-WY  ...                    0

[14790 rows x 427 columns]
           open_covid_region_code region_name hospitalized_new
date                                                          
2020-03-06                  US-AK      Alaska                0
202

In [13]:
# Remove rows with date before 2020-03-09 in the search trends dataset

date = np.arange('2020-01-01', '2020-03-09', dtype='datetime64[D]') 
end_date = np.arange('2020-09-28', np.datetime64('today'), dtype='datetime64[D]')
date = np.concatenate((date, end_date))
date = date.astype(str)

date_to_remove = set(date) & set(df1_temp.index.unique())

df1_temp.drop(index=date_to_remove, axis = 0, inplace = True)
print(df1_temp)

# Reset index
df1_temp = df1_temp.reset_index()
print(df1_temp)



           open_covid_region_code  ... symptom:pancreatitis
date                               ...                     
2020-03-09                  US-AK  ...                    0
2020-03-10                  US-AK  ...                    0
2020-03-11                  US-AK  ...                    0
2020-03-12                  US-AK  ...                 0.59
2020-03-13                  US-AK  ...                    0
...                           ...  ...                  ...
2020-09-23                  US-WY  ...                    0
2020-09-24                  US-WY  ...                    0
2020-09-25                  US-WY  ...                    0
2020-09-26                  US-WY  ...                    0
2020-09-27                  US-WY  ...                    0

[10353 rows x 427 columns]
             date  ... symptom:pancreatitis
0      2020-03-09  ...                    0
1      2020-03-10  ...                    0
2      2020-03-11  ...                    0
3      2020-03-1

In [14]:
# Remove rows with date before 2020-03-09 in the hospitalization dataset
start_date = np.arange('2020-01-01', '2020-03-09', dtype='datetime64[D]')
end_date = np.arange('2020-09-28', np.datetime64('today'), dtype='datetime64[D]')
date1 = np.concatenate((start_date, end_date))
date1 = date1.astype(str)

date1_to_remove = set(date1) & set(df2_temp.index.unique())

df2_temp.drop(index=date1_to_remove, axis = 0, inplace = True)
print(df2_temp)

# Reset index
df2_temp = df2_temp.reset_index()
print(df2_temp)



           open_covid_region_code region_name hospitalized_new
date                                                          
2020-03-09                  US-AK      Alaska                0
2020-03-10                  US-AK      Alaska                0
2020-03-11                  US-AK      Alaska                0
2020-03-12                  US-AK      Alaska                0
2020-03-13                  US-AK      Alaska                0
...                           ...         ...              ...
2020-09-23                  US-WY     Wyoming                0
2020-09-24                  US-WY     Wyoming                2
2020-09-25                  US-WY     Wyoming                1
2020-09-26                  US-WY     Wyoming                2
2020-09-27                  US-WY     Wyoming                4

[10353 rows x 3 columns]
             date open_covid_region_code region_name hospitalized_new
0      2020-03-09                  US-AK      Alaska                0
1      2020-03-

In [15]:
# Merge days to week for daily search trends dataset
df1_array = df1_temp.values
df1_array = np.delete(df1_array, [2,3,4,5], axis=1)
num_rows, num_features = df1_array.shape

weekly_array = []
for index in range(num_rows//7):
  row = [] 
  row.append(df1_array[index*7][0])
  row.append( df1_array[index*7][1])
  for i in range(2, num_features):  
    row.append(np.sum(df1_array[index*7:7+ index*7, i], axis=0))
  weekly_array.append(row)

df1_array = weekly_array

print(df1_array[:10])

[['2020-03-09', 'US-AK', 13.8, 30.419999999999998, 54.349999999999994, 0.0, 4.08, 0.0, 0.0, 34.42, 0.0, 82.14999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 13.739999999999998, 5.38, 0.0, 0.0, 0.0, 54.739999999999995, 0.0, 0.0, 4.22, 0.0, 35.07, 0.0, 8.6, 0.0, 23.47, 0.63, 0.0, 0.0, 20.729999999999997, 0.0, 15.41, 0.0, 36.57, 8.84, 0.0, 0.0, 0.0, 0.0, 0.68, 28.689999999999998, 0.0, 0.64, 13.82, 0.0, 0.0, 0.0, 5.24, 20.599999999999998, 0.0, 0.0, 1.6600000000000001, 1.45, 2.43, 0.0, 8.64, 12.93, 0.0, 0.71, 12.4, 0.0, 0.0, 16.68, 7.450000000000001, 3.3000000000000003, 3.75, 0.0, 0.63, 0.0, 0.0, 10.2, 1.4, 0.0, 7.119999999999999, 3.36, 0.0, 0.0, 0.0, 4.63, 0.0, 277.39, 3.9299999999999997, 0.0, 0.0, 0.0, 8.15, 20.8, 1.51, 89.74999999999999, 0.0, 20.39, 0.0, 0.0, 0.0, 8.219999999999999, 3.06, 13.370000000000001, 0.0, 0.0, 32.269999999999996, 18.71, 0.69, 0.0, 65.65, 0.0, 36.42, 10.990000000000002, 5.390000000000001, 0.0, 0.0, 7.38, 0.0, 0.0, 0.0, 0.0, 0.0, 8.72, 1.12, 5.17, 0.0, 0.0, 0.0, 14.62999999

# Apply Scaling, Merge Array, Save Dataset for Visualization Task

In [16]:
# Merge days to weeks 

# Convert to numpy
df2_array = df2_temp.values 
df2_array = df2_array[:,3:4]

hospitalization_new = [] 
for i in range(len(df2_array) // 7):
  sum = np.sum(df2_array[i*7:7+i*7], axis=0)
  hospitalization_new.append(sum)

hospitalization_new = np.asarray(hospitalization_new)
hos_new = hospitalization_new.clip(min=0)  # remove negative numbers for scaling 
print("Hospitalization shape ", hospitalization_new.shape)

# Get the columns name
headers = list(df1_temp.columns)
headers = headers[:2] + headers[6:]

# Extract all states
states = np.unique(np.asarray(df1_array)[:,1])

# Initialize a dict to store state (key) and the row ranges in the matrix (value)
state_row_ranges = {state: [] for state in states} 
for state in states:
  state_row_ranges[state] = [np.where(np.asarray(df1_array).transpose() == state)[1]]
  state_row_ranges[state] = [state_row_ranges[state][0][0], state_row_ranges[state][0][-1]]

hospitalization_sum = np.sum(hos_new)
# apply Laplace smoothing 
scaling_factor = [np.sum(hos_new[state_row_ranges[state][0] : state_row_ranges[state][1]+1])*1000/hospitalization_sum for state in states]
print("Scaling factor", scaling_factor[:50])

# apply scaling 
for index, row in enumerate(df1_array): 
    key = row[1]
    i_state = np.where(states == key)[0][0]
    s_f = scaling_factor[i_state]
    if (s_f > 0):
        row[2:] = [(i * (s_f)*100) for i in row[2:]]
      
# Save the dataset to .csv file for visualization task
df_visualization = pd.DataFrame(data=df1_array, columns=headers)
df_visualization.to_csv(r'./DailyVisualizationDataWithScale.csv', index=False, header=True)

# Merge dataset 1 and dataset 2
df1_array = df_visualization.values
merged_array = np.append(df1_array, hospitalization_new, axis=1)
print("Merge array shape ", merged_array.shape)



Hospitalization shape  (1479, 1)
Scaling factor [0.08808266068357039, 41.23247216220911, 12.80868690773586, 53.85031330513057, 0.0, 18.399489120568035, 35.30401975008992, 0.0, 0.0, 107.72998749715566, 68.94670265006472, 1.9647326813585282, 0.4037121947996976, 4.431047180498499, 0.0, 30.821591017526003, 6.890021457914838, 12.705923803605028, 0.0, 30.965948711424076, 37.674911366822684, 1.0887995556719117, 0.0, 18.33342712505536, 0.0, 13.718874401466087, 1.6809107747114682, 0.0, 1.9940935682530516, 5.576121769384914, 1.8008010628641056, 57.07511738237906, 8.40455387355734, 0.0, 220.19441800605324, 37.229604582255746, 15.297022072046722, 6.10706447406088, 2.6229058959107623, 6.667368065631369, 22.12587501559797, 3.604048866302755, 20.985693907860643, 0.0, 9.123895602473166, 45.856811848096555, 0.11010332585446297, 18.18172920943365, 17.359624376386996, 0.0024467405745436216]
Merge array shape  (1479, 425)


# Fill Mean in Zero-Value Cells

In [17]:
# This block is trying to get a temp file so that I wont need to run all again
merged_array_copy = merged_array

# Save another copy for caculating the whole column mean
merged_array_copy_col_mean = merged_array_copy.copy()

In [18]:
# Extract all states
states = np.unique(merged_array_copy[:,1])
print(states)

# Initialize a dict to store state (key) and the row ranges in the matrix (value)
state_row_ranges = {state: [] for state in states} 
for state in states:
  state_row_ranges[state] = [np.where(merged_array_copy == state)[0][0]
                             , np.where(merged_array_copy == state)[0][-1]]
print(state_row_ranges)

['US-AK' 'US-AL' 'US-AR' 'US-AZ' 'US-CA' 'US-CO' 'US-CT' 'US-DC' 'US-DE'
 'US-FL' 'US-GA' 'US-HI' 'US-IA' 'US-ID' 'US-IL' 'US-IN' 'US-KS' 'US-KY'
 'US-LA' 'US-MA' 'US-MD' 'US-ME' 'US-MI' 'US-MN' 'US-MO' 'US-MS' 'US-MT'
 'US-NC' 'US-ND' 'US-NE' 'US-NH' 'US-NJ' 'US-NM' 'US-NV' 'US-NY' 'US-OH'
 'US-OK' 'US-OR' 'US-PA' 'US-RI' 'US-SC' 'US-SD' 'US-TN' 'US-TX' 'US-UT'
 'US-VA' 'US-VT' 'US-WA' 'US-WI' 'US-WV' 'US-WY']
{'US-AK': [0, 28], 'US-AL': [29, 57], 'US-AR': [58, 86], 'US-AZ': [87, 115], 'US-CA': [116, 144], 'US-CO': [145, 173], 'US-CT': [174, 202], 'US-DC': [203, 231], 'US-DE': [232, 260], 'US-FL': [261, 289], 'US-GA': [290, 318], 'US-HI': [319, 347], 'US-IA': [348, 376], 'US-ID': [377, 405], 'US-IL': [406, 434], 'US-IN': [435, 463], 'US-KS': [464, 492], 'US-KY': [493, 521], 'US-LA': [522, 550], 'US-MA': [551, 579], 'US-MD': [580, 608], 'US-ME': [609, 637], 'US-MI': [638, 666], 'US-MN': [667, 695], 'US-MO': [696, 724], 'US-MS': [725, 753], 'US-MT': [754, 782], 'US-NC': [783, 811], 'US-

In [19]:
# Get new matrixs
# Split the old matrix based on the states
# New matrix only contains one state info
state_matrices = {state: [] for state in states} 
for key in state_row_ranges:
  new_matrix = merged_array_copy[state_row_ranges[key][0] : state_row_ranges[key][1]+1]
  state_matrices[key] = new_matrix

In [20]:
# Recreate header file
headers = list(df1_temp.columns)
headers.append('hospitalized_new')
headers = headers[:2] + headers[6:]

print(len(headers))

# Reshape the header to fit the new matrix storing in state_matrices
headers = np.expand_dims(np.array(headers), axis=0)


425


In [21]:
# Save a copy for get the whole column mean
state_matrices_copy = state_matrices
headers_copy = headers.copy()

# Create a new matrix for saving all the data (Refill the 0 with state mean)
matrices_list = []

In [22]:
# Find some state to drop
region_to_be_dropped = ['US-VT']
symptom_to_be_dropped = []

In [23]:
# Calculate the mean for the cell contains 0
for key in state_matrices:
  # Split the matrix and get the matrix containing only floats to get the mean
  values = np.hsplit(state_matrices[key], [2])
  if key in region_to_be_dropped:
    continue
 
  # Get the mean for different states with different symptoms 

  ### Fill Mean 
  means = np.mean(values[1], axis=0)

  values[1] = values[1].transpose()
  num_values, _ = values[1].shape
  for i in range(num_values-1):   # Do not apply mean for the hospitalization case, so it should be num_values -1 
      values[1][i][values[1][i] == 0] = means[i]

  values[1] = values[1].transpose()

  # Add back all the rows to the dictionary 
  state_matrices[key] = np.append(values[0], values[1], axis=1)
  matrices_list.append(state_matrices[key])
  state_matrices[key] = np.append(headers, state_matrices[key], axis=0)


# Save Dataset For Supervised Learning Task

In [24]:
# Combine matrices together and add the header
total_data = matrices_list[0]

# Drop the first one
for matrix in matrices_list[1:]:
    total_data = np.append(matrix, total_data, axis=0)
print(total_data.shape)

# A threshold to drop symptoms 
threshold = 100 
sums = np.sum(total_data[:,2:], axis = 0)
symptom_to_be_dropped = np.where(sums < threshold)

symptom_ids = []
total_data = np.append(headers, total_data, axis=0)
print(total_data.shape)

# Convert numpy matrix to data frame
df = pd.DataFrame(data=total_data)
df = df.set_index(1)
for symptom in symptom_to_be_dropped:
    symptom_ids.append(headers[0][2+symptom])
print("Symptoms to drop: ", list(symptom_ids[0]))
df = df.drop(columns=list(2 + symptom_to_be_dropped[0]),axis=1)

print(df.shape)

# Save dataset to a csv file for task 3 supervised learning 
df.to_csv(r'./AllDataWithStateMeanDailyWithScale.csv', index=True, header=False)

(1450, 425)
(1451, 425)
Symptoms to drop:  []
(1451, 424)
